In [2]:
import pygame 
import sys
import random
import psycopg2
import datetime

def get_or_create_user(username):
    conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)
    cur = conn.cursor()
    cur.execute("SELECT id FROM game_user WHERE username = %s", (username,))
    result = cur.fetchone()
    if result:
        user_id = result[0]
    else:
        cur.execute("INSERT INTO game_user (username) VALUES (%s) RETURNING id", (username,))
        user_id = cur.fetchone()[0]
        conn.commit()
    cur.close()
    conn.close()
    return user_id

def get_user_level(user_id):
    conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)
    cur = conn.cursor()
    cur.execute("SELECT level FROM user_score WHERE user_id = %s ORDER BY created_at DESC LIMIT 1;", (user_id,))
    result = cur.fetchone()
    cur.close()
    conn.close()
    return result[0] if result else 1

def save_game(user_id, level, score):
    conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)
    cur = conn.cursor()
    cur.execute("INSERT INTO user_score (user_id, level, score) VALUES (%s, %s, %s);", (user_id, level, score))
    conn.commit()
    cur.close()
    conn.close()

username = input("Введите имя пользователя: ").strip()
user_id = get_or_create_user(username)
level = get_user_level(user_id)
speed = 10 + (level - 1) * 2

pygame.init()
width, height = 500, 500
cell_size = 10
black = (0, 0, 0)
green = (0, 255, 0)
red = (255, 0, 0)
white = (255, 255, 255)
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Snake Game')
snake_pos = [100, 100]
snake_body = [[100, 100], [90, 100], [80, 100]]
direction = 'RIGHT'
change_to = direction
food_pos = [random.randrange(1, width // cell_size) * cell_size,
            random.randrange(1, height // cell_size) * cell_size]
food_weight = 1 
food_timer = 0  
food_lifetime = 100 
score = 0
font = pygame.font.SysFont('Verdana', 18)
clock = pygame.time.Clock()

def is_collision(pos):
    if pos in snake_body[1:]:  
        return True
    if pos[0] < 0 or pos[0] >= width or pos[1] < 0 or pos[1] >= height: 
        return True
    return False

def generate_food():
    while True:
        new_pos = [random.randrange(1, width // cell_size) * cell_size,
                   random.randrange(1, height // cell_size) * cell_size]
        if new_pos not in snake_body:
            return new_pos

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            save_game(user_id, level, score)
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and direction != 'DOWN':
                change_to = 'UP'
            elif event.key == pygame.K_DOWN and direction != 'UP':
                change_to = 'DOWN'
            elif event.key == pygame.K_LEFT and direction != 'RIGHT':
                change_to = 'LEFT'
            elif event.key == pygame.K_RIGHT and direction != 'LEFT':
                change_to = 'RIGHT'
            elif event.key == pygame.K_p:
                save_game(user_id, level, score)
                paused = True
                while paused:
                    for ev in pygame.event.get():
                        if ev.type == pygame.KEYDOWN and ev.key == pygame.K_p:
                            paused = False

    direction = change_to
    if direction == 'UP':
        snake_pos[1] -= cell_size
    elif direction == 'DOWN':
        snake_pos[1] += cell_size
    elif direction == 'LEFT':
        snake_pos[0] -= cell_size
    elif direction == 'RIGHT':
        snake_pos[0] += cell_size

    if is_collision(snake_pos):
        save_game(user_id, level, score)
        running = False

    snake_body.insert(0, list(snake_pos))
    if snake_pos == food_pos:
        score += food_weight
        if score % 4 == 0:
            level += 1
            speed += 2
        food_pos = generate_food()
        food_weight = random.choice([1, 2, 3])
        food_timer = 0
    else:
        snake_body.pop()

    food_timer += 1
    if food_timer > food_lifetime:
        food_pos = generate_food()
        food_weight = random.choice([1, 2, 3])
        food_timer = 0

    screen.fill(black)
    for block in snake_body:
        pygame.draw.rect(screen, green, pygame.Rect(block[0], block[1], cell_size, cell_size))
    pygame.draw.rect(screen, red, pygame.Rect(food_pos[0], food_pos[1], cell_size, cell_size))
    score_text = font.render(f'Score: {score}', True, white)
    level_text = font.render(f'Level: {level}', True, white)
    screen.blit(score_text, (10, 10))
    screen.blit(level_text, (400, 10))
    pygame.display.flip()
    clock.tick(speed)

pygame.quit()


In [3]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres",
    password="80950644dR?)", 
    host="localhost",
    port="9956"
)

cur = conn.cursor()

cur.execute('''
    SELECT u.username, s.level, s.score, s.created_at
    FROM game_user u
    JOIN user_score s ON u.id = s.user_id
    ORDER BY s.created_at DESC;
''')

rows = cur.fetchall()
for row in rows:
    print(f"Пользователь: {row[0]}, Уровень: {row[1]}, Счёт: {row[2]}, Дата: {row[3]}")

cur.close()
conn.close()


Пользователь: user1, Уровень: 2, Счёт: 4, Дата: 2025-04-18 23:17:29.698157
